# CHAPTER10 특성선택을 사용한 차원 축소

## 10.1 분산을 기준으로 수치 특성 선택하기

### **VT(Variace thresholding) : 분산기준설정**

https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.VarianceThreshold.html#sklearn.feature_selection.VarianceThreshold
* 분산이 높은 특성보다 분산이 낮은 특성이 효과적이거나 유용하지 않다는 idea에 기반
* 각 먼저 특성의 분산을 계산,그다음 분산이 기준값에 미치지 못하는 모든 특성을 삭제
* threshold매개변수 : 이 임계값보다 낮은 분산을 가진 특성은 제거된다. 기본값은 0으로 모든 특성을 선택하는 것, 즉 0이 아닌 분산으로 유지, 모든 표본에서 동일한 값을 가지는 특성을 제거한다.



In [4]:
from sklearn import datasets
from sklearn.feature_selection import VarianceThreshold

iris = datasets.load_iris()

features=iris.data
target=iris.target

#기준값 생성
thresholder=VarianceThreshold(threshold=.5)

#기준값 보다 높은 특성 선택
features_high_variance=thresholder.fit_transform(features)

features_high_variance[0:3]


array([[5.1, 1.4, 0.2],
       [4.9, 1.4, 0.2],
       [4.7, 1.3, 0.2]])

In [5]:
features[0:3]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2]])

1. 분산은 원점에 맞춰진 값이 아니다
    : 특성의 제곱 단위이다. 그러므로 특성의 단위가 다르면 VT는 작동하지 않음 EX)각 특성의 단위가 년, 원 단위인 경우
2. 분산의 기준값을 수동으로 선택하기 때문에 어떤 값이 좋은지 판단할 수 있어야 함

In [6]:
thresholder.variances_
#각 특성의 분산을 확인할 수 있음

array([0.68112222, 0.18871289, 3.09550267, 0.57713289])

주의! 특성이 평균이 0이고 단위분산으로 표준화 돼있으면 vt가 올바르게 작동하지 않음

In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
features_std=scaler.fit_transform(features)

selector=VarianceThreshold()
selector.fit(features_std).variances_

array([1., 1., 1., 1.])

***

## 10.2 분산을 기준으로 이진 특성 선택하기

이진 범수형 특성에서 분산이 낮은 특성을 삭제하기위함

### Bernoulli 확률 변수
* 변수의 분산이 기준값 이상인 특성을 선택
* <math xmlns="http://www.w3.org/1998/Math/MathML" display="block">
  <mrow>
    <mi mathvariant="normal">V</mi>
    <mi mathvariant="normal">a</mi>
    <mi mathvariant="normal">r</mi>
  </mrow>
  <mo stretchy="false">[</mo>
  <mi>X</mi>
  <mo stretchy="false">]</mo>
  <mo>=</mo>
  <mi>p</mi>
  <mo stretchy="false">(</mo>
  <mn>1</mn>
  <mo>&#x2212;</mo>
  <mi>p</mi>
  <mo stretchy="false">)</mo>
</math>
    - 아래 예제에서 p는 클래스 1의 샘플 비율

In [14]:
from sklearn.feature_selection import VarianceThreshold

# 특성 0 : 80% 클래스 0
# 특성 1 : 80% 클래스 1
# 특성 3: 60%가 클래스 0
feature=[[0,1,0],
        [0,1,1],
        [0,1,0],
        [0,1,1],
        [1,0,0]]

#분산을 기준으로 선택
threshold=VarianceThreshold(threshold=(.75*(1-.75)))
threshold.fit_transform(feature)

array([[0],
       [1],
       [0],
       [1],
       [0]])

* 이진 특성의 경우 variance_속성에서 분산 확인 가능

In [15]:
threshold.variances_

array([0.16, 0.16, 0.24])

* variancethreshold클래스는 수치 특성, 이진 특성에 상관없이 넘파이 var함수를 사용해 분산을 계산 

In [11]:
import numpy as np
np.var(feature, axis=0)

array([0.16, 0.16, 0.24])

* 이진 특성에서 var함수를 사용하는 것은 베르누이 확률 변수의 분산과 같기 때문

***

## 10.3 상관관계가 큰 특성 다루기

### 상관관계 행렬

* 상관관계가 큰 특성을 확인하고 이들 중 하나를 삭제한다
* corr() : 상관관계 행렬 생성   
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html#pandas.DataFrame.corr

* np.triu함수는 주어진 배열에서 상삼각 행렬을 추출해 반환
    - k매개변수가 기본값0일 경우, 반환되는 행렬에 대각원소가 포함됨
    - k 값이 커질수록 대각원소에서 k만큼 떨어진 삼각행렬을 반환

In [51]:
import numpy as np
import pandas as pd

features=np.array([[1,1,1],
                  [2,2,0],
                  [3,3,1],
                  [4,4,0],
                  [5,5,1],
                  [6,6,0],
                  [7,7,1],
                  [8,7,0],
                  [9,7,1]])

df=pd.DataFrame(features)

# 상관관계 행렬을 만듬
corr_matrix=df.corr()

# 상관관계의 상삼각 행렬을 선택 
# https://ko.wikipedia.org/wiki/%EC%82%BC%EA%B0%81%ED%96%89%EB%A0%AC
upper=corr_matrix.where(np.triu(np.ones(corr_matrix.shape),k=1).astype(np.bool))

#상관관계가 0.95보다 큰 특성의 열을 찾는다
to_drop=[column for column in upper.columns if any(upper[column]>0.95)]

#특성 제거
df.drop(df.columns[to_drop],axis=1).head(3)

,0,2
0,1,1
1,2,0
2,3,1


In [58]:
corr_matrix


,0,1,2
0,1.000000,0.976103,0.000000
1,0.976103,1.000000,-0.034503
2,0.000000,-0.034503,1.000000


In [22]:
df.corr()

,0,1,2
0,1.000000,0.976103,0.000000
1,0.976103,1.000000,-0.034503
2,0.000000,-0.034503,1.000000


In [53]:
# 상관관계 행렬의 상삼각 행렬
upper

,0,1,2
0,NaN,0.976103,0.000000
1,NaN,NaN,-0.034503
2,NaN,NaN,NaN


# numpy의 corrcoef함수로 상관관계 행렬 구하기

* 특성이 행에 놓여있음
* rowvar매개변수를 False로 지정하면 특성을 열에 놓게 됨

In [54]:
np.corrcoef(features, rowvar=False)


array([[ 1.        ,  0.97610336,  0.        ],
       [ 0.97610336,  1.        , -0.03450328],
       [ 0.        , -0.03450328,  1.        ]])

* np.triu함수는 주어진 배열에서 상삼각 행렬을 추출해 반환
    - k매개변수가 기본값0일 경우, 반환되는 행렬에 대각원소가 포함됨
    - k 값이 커질수록 대각원소에서 k만큼 떨어진 삼각행렬을 반환

In [55]:
np.triu(np.ones((4,4)),k=2)


array([[0., 0., 1., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [57]:
np.triu(np.ones((4,4)),k=0)

array([[1., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 0., 1., 1.],
       [0., 0., 0., 1.]])

* np.tril함수는 하삼각 행렬을 구함

In [56]:
np.tril(np.ones((4,4)),k=0)

array([[1., 0., 0., 0.],
       [1., 1., 0., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 1.]])

***

## 10.4 분류 작업에 관련 없는 특성 삭제하기

In [25]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif

# 데이터를 로드합니다.
iris = load_iris()
features = iris.data
target = iris.target

# 범주형 데이터를 정수형으로 변환합니다.
features = features.astype(int)

# 카이제곱 통계값이 가장 큰 특성 두 개를 선택합니다.
chi2_selector = SelectKBest(chi2, k=2)
features_kbest = chi2_selector.fit_transform(features, target)

# 결과를 확인합니다.
print("원본 특성 개수:", features.shape[1])
print("줄어든 특성 개수:", features_kbest.shape[1])

원본 특성 개수: 4
줄어든 특성 개수: 2


In [27]:
#F-값이 가장 높은 특성 두 개를 선택합니다.
fvalue_selector = SelectKBest(f_classif, k=2)
features_kbest = fvalue_selector.fit_transform(features, target)

# 결과를 확인합니다.
print("원본 특성 개수:", features.shape[1])
print("줄어든 특성 개수:", features_kbest.shape[1])

원본 특성 개수: 4
줄어든 특성 개수: 2


In [28]:
# 라이브러리를 임포트합니다.
from sklearn.feature_selection import SelectPercentile

# 가장 큰 F-값의 상위 75% 특성을 선택합니다.
fvalue_selector = SelectPercentile(f_classif, percentile=75)
features_kbest = fvalue_selector.fit_transform(features, target)

# 결과를 선택합니다.
print("원본 특성 개수:", features.shape[1])
print("줄어든 특성 개수:", features_kbest.shape[1])

원본 특성 개수: 4
줄어든 특성 개수: 3


In [29]:
target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [30]:

observed = np.sum(features.reshape(3, 50, 4), axis=1)
observed

array([[230, 152,  50,   0],
       [274, 116, 191,  50],
       [304, 129, 255,  79]])

In [31]:
expected = features.sum(axis=0) / 3
expected

array([269.33333333, 132.33333333, 165.33333333,  43.        ])

In [32]:
np.sum((observed - expected)**2 / expected, axis=0)

array([ 10.28712871,   5.02267003, 133.06854839,  74.27906977])

In [33]:

chi2_selector.scores_

array([ 10.28712871,   5.02267003, 133.06854839,  74.27906977])

In [34]:

total_mean = np.mean(features, axis=0)
total_mean

array([5.38666667, 2.64666667, 3.30666667, 0.86      ])

In [35]:
class_mean = np.mean(features.reshape(3, 50, 4), axis=1)
class_mean

array([[4.6 , 3.04, 1.  , 0.  ],
       [5.48, 2.32, 3.82, 1.  ],
       [6.08, 2.58, 5.1 , 1.58]])

In [36]:
ss_between = np.sum(50 * (class_mean - total_mean)**2, axis=0)
ss_between

array([ 55.41333333,  13.29333333, 440.01333333,  63.88      ])

In [37]:
ss_total = np.sum((features - total_mean)**2, axis=0)
ss_total

array([105.57333333,  42.27333333, 467.89333333,  76.06      ])

In [38]:
f = (ss_between/(3-1)) / ((ss_total-ss_between)/(150-3))
f


array([  81.19776715,   33.71497585, 1160.00645624,  385.48275862])

In [39]:

fvalue_selector.scores_

array([  81.19715 ,   33.715004, 1160.0116  ,  385.483   ], dtype=float32)

***

## 10.5 재귀적 특성 제거하기

In [40]:
# 라이브러리를 임포트합니다.
from sklearn.datasets import make_regression
from sklearn.feature_selection import RFECV
from sklearn import datasets, linear_model

# 특성 행렬과 타깃 벡터를 생성합니다.
features, target = make_regression(n_samples = 10000,
                                   n_features = 100,
                                   n_informative = 2,
                                   random_state = 1)

# 선형 회귀 모델을 만듭니다.
ols = linear_model.LinearRegression()

# 재귀적으로 특성을 제거합니다.
rfecv = RFECV(estimator=ols, step=1, scoring="neg_mean_squared_error")
rfecv.fit(features, target)
rfecv.transform(features)

C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


array([[ 0.00850799,  0.7031277 ],
       [-1.07500204,  2.56148527],
       [ 1.37940721, -1.77039484],
       ...,
       [-0.80331656, -1.60648007],
       [ 0.39508844, -1.34564911],
       [-0.55383035,  0.82880112]])

In [41]:
# 최선의 특성 개수
rfecv.n_features_

2

In [42]:

# 선택된 특성이 표시된 불리언 마스크
rfecv.support_

array([False, False, False, False, False,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False])

In [43]:

# 특성의 순위: 최고(1)에서 최악(96)까지
rfecv.ranking_

array([82, 84, 74, 33, 81,  1, 18, 46, 57, 67, 45,  7, 58, 52, 78,  8,  5,
       73, 31, 11, 43, 14, 34, 83, 21, 96, 20, 41, 94, 90, 71, 47, 30, 27,
       89, 50, 25, 69, 86,  1, 76, 19, 97, 88,  9, 16, 23, 80, 75, 54, 91,
       12, 65, 59, 24, 32,  4, 26, 10, 42, 72,  2, 87, 40, 66,  3, 92, 17,
       39, 35, 13, 79, 38,  6, 53, 60, 22, 61, 28, 95, 93, 36, 99, 48, 51,
       68, 37, 70, 15, 98, 56, 29, 44, 63, 49, 64, 77, 85, 55, 62])

In [44]:
from sklearn.feature_selection import RFE

rfe = RFE(estimator=ols, n_features_to_select=3)
rfe.fit(features, target)
rfe.transform(features)

array([[ 0.00850799,  0.7031277 , -1.16432076],
       [-1.07500204,  2.56148527, -1.3936433 ],
       [ 1.37940721, -1.77039484,  1.02058188],
       ...,
       [-0.80331656, -1.60648007, -0.64067478],
       [ 0.39508844, -1.34564911, -1.91468325],
       [-0.55383035,  0.82880112,  0.27936745]])

In [45]:
np.all(rfe.support_ == rfecv.support_)

False